<a href="https://colab.research.google.com/github/Abidt2002/AI-Resume-Builder/blob/main/AI_Powered_Resume_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary packages
!pip install nbformat nbconvert nbstripout --quiet

import nbformat
from nbconvert.preprocessors import ClearOutputPreprocessor

# Path to your NeuroCare-AI notebook
notebook_path = '/content/drive/MyDrive/Colab Notebooks/AI Powered Resume App.ipynb'

# Load the notebook
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Clear outputs
clear_output = ClearOutputPreprocessor()
nb, _ = clear_output.preprocess(nb, {})

# Remove widgets metadata if exists
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Save cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print(f"✅ Notebook '{notebook_path}' is now GitHub-safe!")


✅ Notebook '/content/drive/MyDrive/Colab Notebooks/AI Powered Resume App.ipynb' is now GitHub-safe!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip -q install faiss-cpu sentence-transformers pandas gradio openai


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.1 MB/s eta 0:00:00


In [3]:
import os, zipfile, glob

zip_path = "/content/drive/MyDrive/AI Powered Resume Dataset.zip"
extract_path = "/content/resume_dataset"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

# Try to find dice_com-job_us_sample.csv anywhere inside
matches = glob.glob(os.path.join(extract_path, "**", "dice_com-job_us_sample.csv"), recursive=True)
if not matches:
    raise FileNotFoundError("dice_com-job_us_sample.csv not found inside the ZIP.")
csv_path = matches[0]
csv_path


'/content/resume_dataset/dice_com-job_us_sample.csv'

In [4]:
import pandas as pd
import numpy as np

# Robust CSV loader (handles common encodings/quoting)
try:
    df = pd.read_csv(csv_path, low_memory=False)
except UnicodeDecodeError:
    df = pd.read_csv(csv_path, low_memory=False, encoding="latin-1")

# Keep only the fields we care about (and coerce to string)
expected_cols = [
    "advertiserurl","company","employmenttype_jobstatus","jobdescription",
    "joblocation_address","jobtitle","postdate","shift","skills"
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns in CSV: {missing}")

for c in expected_cols:
    df[c] = df[c].astype(str).replace(["nan","None","NaN"], "")

# Basic cleaning
df["jobtitle"] = df["jobtitle"].str.strip()
df["skills"] = df["skills"].str.replace(r"\s+", " ", regex=True).str.strip()
df["jobdescription"] = df["jobdescription"].str.replace(r"\s+", " ", regex=True).str.strip()

# Parse dates if available
df["postdate_parsed"] = pd.to_datetime(df["postdate"], errors="coerce")

# Optional: drop obvious duplicates on (jobtitle, company, location)
df.drop_duplicates(subset=["jobtitle","company","joblocation_address"], inplace=True, keep="first")

df[expected_cols].head(3)


/tmp/ipython-input-1170991774.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["postdate_parsed"] = pd.to_datetime(df["postdate"], errors="coerce")


,advertiserurl,company,employmenttype_jobstatus,jobdescription,joblocation_address,jobtitle,postdate,shift,skills
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,SEE BELOW
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,"linux/unix, network monitoring, incident respo..."
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...","Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,"Enterprise Solutions Architecture, business in..."


In [5]:
print(df.columns)


Index(['advertiserurl', 'company', 'employmenttype_jobstatus',
       'jobdescription', 'jobid', 'joblocation_address', 'jobtitle',
       'postdate', 'shift', 'site_name', 'skills', 'uniq_id',
       'postdate_parsed'],
      dtype='object')


In [6]:
# Create a combined text column for embeddings
df["combined_text"] = (
    df["jobtitle"].fillna('') + " | " +
    df["jobdescription"].fillna('') + " | " +
    df["skills"].fillna('')
)


In [7]:
print(df[["jobtitle", "skills", "combined_text"]].head(3))


                        jobtitle  \
0       AUTOMATION TEST ENGINEER   
1  Information Security Engineer   
2   Business Solutions Architect   

                                              skills  \
0                                          SEE BELOW   
1  linux/unix, network monitoring, incident respo...   
2  Enterprise Solutions Architecture, business in...   

                                       combined_text  
0  AUTOMATION TEST ENGINEER | Looking for Seleniu...  
1  Information Security Engineer | The University...  
2  Business Solutions Architect | GalaxE.Solution...  


In [8]:
texts = df["combined_text"].tolist()


In [9]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load smaller & faster model
embedder = SentenceTransformer("paraphrase-MiniLM-L3-v2")

# Convert job descriptions into a list of text
texts = df["jobdescription"].astype(str).tolist()

# Encode all job descriptions
embeddings = embedder.encode(
    texts,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=16   # Small batch size for speed
)

# Convert to numpy array
embeddings = np.array(embeddings)

print("✅ Embeddings created successfully!")
print("Shape:", embeddings.shape)  # (num_samples, 384)
from sentence_transformers import SentenceTransformer
import numpy as np

# Load smaller & faster model
embedder = SentenceTransformer("paraphrase-MiniLM-L3-v2")

# Convert job descriptions into a list of text
texts = df["jobdescription"].astype(str).tolist()

# Encode all job descriptions
embeddings = embedder.encode(
    texts,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=16   # Small batch size for speed
)

# Convert to numpy array
embeddings = np.array(embeddings)

print("✅ Embeddings created successfully!")
print("Shape:", embeddings.shape)  # (num_samples, 384)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1274 [00:00<?, ?it/s]

✅ Embeddings created successfully!
Shape: (20372, 384)


Batches:   0%|          | 0/1274 [00:00<?, ?it/s]

✅ Embeddings created successfully!
Shape: (20372, 384)


In [10]:
import faiss
import numpy as np

# Build FAISS index (Inner Product similarity)
index = faiss.IndexFlatIP(embeddings.shape[1])

# Add embeddings to the index
index.add(embeddings)

print("Index size:", index.ntotal)


Index size: 20372


In [11]:
# Example query text (a candidate resume)
query_text = "Python developer with experience in machine learning, NLP, and cloud deployment."

# Embed query
query_emb = embedder.encode([query_text], normalize_embeddings=True)

# Search top 5 similar jobs
k = 5
distances, indices = index.search(query_emb, k)

print("\nTop Matches:")
for i, idx in enumerate(indices[0]):
    print(f"Rank {i+1}: {df.iloc[idx]['jobtitle']} | {df.iloc[idx]['company']}")
    print(f"Score: {distances[0][i]:.4f}")
    print(f"Description: {df.iloc[idx]['jobdescription'][:150]}...\n")



Top Matches:
Rank 1: Python Developer | The Armada Group
Score: 0.6587
Description: We are looking for a Python Developer to be a part of a leading companies innovation group.Minimum Qualifications:Strong Python programming skills, wi...

Rank 2: Software Engineer | Matlen Silver
Score: 0.6529
Description: Python Developer (someone who has Python experience or is willing to learn Python as this person will be developing in Python)Background and Responsib...

Rank 3: Java/AWS Developer | Peterson Technology Partners
Score: 0.5956
Description: .Cloud Platform Engineer/Consultant (AWS Environment Management)The Cloud Platform Engineer/Consultant is responsible for building and operating the c...

Rank 4: In person interview for Developer(MEMSQL+Python+Hadoop exp) | US IT Solutions, Inc.
Score: 0.5910
Description: We are looking for Software developer with hadoop and Python experience in Philadelphia PA. Mode of Interview: F2FPhiladelphia, PA. Backend developmen...

Rank 5: Python Develop

In [13]:
import faiss

# Create FAISS index (Inner Product similarity)
index = faiss.IndexFlatIP(embeddings.shape[1])

# Add embeddings
index.add(embeddings)

print("Index size:", index.ntotal)


Index size: 20372


In [15]:
import pickle, os

SAVE_DIR = "/content/resume_rag"
os.makedirs(SAVE_DIR, exist_ok=True)

# Save
faiss.write_index(index, os.path.join(SAVE_DIR, "job_index.faiss"))
with open(os.path.join(SAVE_DIR, "job_df.pkl"), "wb") as f:
    pickle.dump(df, f)

# # Reload example
# index = faiss.read_index(os.path.join(SAVE_DIR, "job_index.faiss"))
# df = pickle.load(open(os.path.join(SAVE_DIR, "job_df.pkl"), "rb"))


In [16]:
def _filter_df(df_in, location=None, emp_type=None, shift=None, days_recent=None):
    m = pd.Series(True, index=df_in.index)
    if location:
        m &= df_in["joblocation_address"].str.contains(location, case=False, na=False)
    if emp_type:
        m &= df_in["employmenttype_jobstatus"].str.contains(emp_type, case=False, na=False)
    if shift:
        m &= df_in["shift"].str.contains(shift, case=False, na=False)
    if days_recent and "postdate_parsed" in df_in:
        cutoff = pd.Timestamp.utcnow() - pd.Timedelta(days=int(days_recent))
        m &= df_in["postdate_parsed"] >= cutoff
    return df_in[m]

def search_jobs(query, top_k=8, location=None, emp_type=None, shift=None, days_recent=None):
    """
    Retrieves top_k jobs for a text query. Filters are simple contains checks.
    """
    # oversample globally, then apply filters on the picked list
    q_vec = embedder.encode([query], normalize_embeddings=True).astype("float32")
    D, I = index.search(q_vec, min(top_k*20, len(df)))  # oversample for better post-filtering

    # Build candidate list
    candidates = []
    for rank, (i, score) in enumerate(zip(I[0], D[0]), start=1):
        row = df.iloc[i]
        candidates.append({
            "idx": int(i),
            "score": float(score),
            "jobtitle": row["jobtitle"],
            "company": row["company"],
            "location": row["joblocation_address"],
            "skills": row["skills"],
            "employment_type": row["employmenttype_jobstatus"],
            "shift": row["shift"],
            "postdate": row["postdate"],
            "description": (row["jobdescription"][:500] + "...") if len(row["jobdescription"])>500 else row["jobdescription"],
            "link": row["advertiserurl"],
        })

    # apply filters on candidates
    def _ok(c):
        ok = True
        if location:
            ok &= (location.lower() in (c["location"] or "").lower())
        if emp_type:
            ok &= (emp_type.lower() in (c["employment_type"] or "").lower())
        if shift:
            ok &= (shift.lower() in (c["shift"] or "").lower())
        if days_recent:
            try:
                pd_date = pd.to_datetime(c["postdate"], errors="coerce", utc=True)
                cutoff = pd.Timestamp.utcnow().tz_localize("UTC") - pd.Timedelta(days=int(days_recent))
                ok &= (pd_date >= cutoff)
            except Exception:
                pass
        return ok

    filtered = [c for c in candidates if _ok(c)]
    # take top_k by similarity score
    filtered = sorted(filtered, key=lambda x: x["score"], reverse=True)[:top_k]
    return filtered


In [17]:
# 1) Install required packages (run this cell)
!pip -q install pypdf PyPDF2 docx2txt

# 2) Robust resume extraction functions (run this cell after install)
import io, os
import docx2txt

def extract_text_from_pdf(path_or_bytes):
    """
    Accepts either a file path (str) or bytes. Returns extracted text or empty string.
    Tries pypdf first, falls back to PyPDF2.
    """
    try:
        # try pypdf
        from pypdf import PdfReader
        if isinstance(path_or_bytes, (bytes, bytearray)):
            reader = PdfReader(io.BytesIO(path_or_bytes))
        else:
            reader = PdfReader(path_or_bytes)
        pages = [p.extract_text() or "" for p in reader.pages]
        return "\n".join(pages).strip()
    except Exception:
        # fallback to PyPDF2
        try:
            from PyPDF2 import PdfReader as PDF2Reader
            if isinstance(path_or_bytes, (bytes, bytearray)):
                reader = PDF2Reader(io.BytesIO(path_or_bytes))
            else:
                reader = PDF2Reader(path_or_bytes)
            pages = []
            for p in reader.pages:
                try:
                    pages.append(p.extract_text() or "")
                except Exception:
                    pages.append("")
            return "\n".join(pages).strip()
        except Exception:
            return ""

def extract_text_from_docx(path_or_bytes):
    """
    Accepts either a file path (str) or bytes. Uses docx2txt (writes a temp file for bytes).
    """
    try:
        if isinstance(path_or_bytes, (bytes, bytearray)):
            tmp = f"/tmp/resume_{os.getpid()}.docx"
            with open(tmp, "wb") as f:
                f.write(path_or_bytes)
            text = docx2txt.process(tmp) or ""
            try: os.remove(tmp)
            except Exception: pass
            return text.strip()
        else:
            text = docx2txt.process(path_or_bytes) or ""
            return text.strip()
    except Exception:
        return ""

def extract_resume_text(path_or_bytes):
    """
    Generic wrapper — path_or_bytes can be a filepath string (Gradio type='filepath') or raw bytes.
    """
    if not path_or_bytes:
        return ""
    # If it's a path string, check extension
    if isinstance(path_or_bytes, str):
        lower = path_or_bytes.lower()
        if lower.endswith(".pdf"):
            return extract_text_from_pdf(path_or_bytes)
        if lower.endswith(".docx"):
            return extract_text_from_docx(path_or_bytes)
        # Try reading as file anyway
        try:
            with open(path_or_bytes, "rb") as f:
                data = f.read()
            # try PDF first
            txt = extract_text_from_pdf(data)
            if txt.strip():
                return txt
            return extract_text_from_docx(data)
        except Exception:
            return ""
    else:
        # assume bytes-like
        data = path_or_bytes
        txt = extract_text_from_pdf(data)
        if txt.strip():
            return txt
        return extract_text_from_docx(data)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.3 MB/s eta 0:00:00


In [18]:
import gradio as gr

# --- RAG query function (already defined earlier but re-define here to avoid NameError) ---
def rag_query(query, top_k=5):
    """Search jobs by user text query using FAISS + LLM."""
    q_emb = embedder.encode([query], normalize_embeddings=True)
    scores, idx = index.search(np.array(q_emb, dtype="float32"), top_k)
    results = []
    for i in idx[0]:
        row = df.iloc[i]
        results.append(f"**{row['jobtitle']}** at *{row['company']}* — {row['joblocation_address']}\n\n{row['jobdescription'][:300]}...")
    return "\n\n".join(results)

# --- Resume matcher ---
def match_resume(file):
    """Take uploaded resume (PDF/DOCX), extract text, search jobs."""
    try:
        resume_text = extract_resume_text(file)
        if not resume_text.strip():
            return "⚠️ Could not extract text from resume."
        return rag_query(resume_text, top_k=5)
    except Exception as e:
        return f"Error: {e}"

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# 📄 AI Powered Resume & Job Matcher")
    gr.Markdown("Search jobs by query OR upload a resume to get matching jobs.")

    with gr.Tab("🔍 Search by Query"):
        query_input = gr.Textbox(label="Enter skills, job title, or keywords")
        query_output = gr.Markdown()
        query_btn = gr.Button("Find Jobs")
        query_btn.click(fn=rag_query, inputs=query_input, outputs=query_output)

    with gr.Tab("📂 Upload Resume"):
        resume_file = gr.File(type="filepath", file_types=[".pdf", ".docx"], label="Upload Resume")
        resume_output = gr.Markdown()
        resume_btn = gr.Button("Match Jobs")
        resume_btn.click(fn=match_resume, inputs=resume_file, outputs=resume_output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4303ca863a344ff45.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
